In [6]:
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

# Paths (adjust if running from a different cwd)
EXP_DIR = Path('/Users/juandiegoyoung/Documents/Price_action_and_indicators/experiments') / 'boschonk' / 'BTCUSDT' / '1h' / 'window_5' / 'rr_1.0'
DATA_DIR = Path('/Users/juandiegoyoung/Documents/Price_action_and_indicators/candle_data') / 'BTCUSDT' / '1h'
CANONICAL_DIR = EXP_DIR / 'canonical_output'
OUT_DIR = EXP_DIR / 'notebooks'
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Load candles
csvs = sorted([p for p in DATA_DIR.glob('*.csv')])
if not csvs:
    raise SystemExit(f'No candle CSVs in {DATA_DIR}')
btc = pd.read_csv(csvs[0])

if 'open_time' in btc.columns:
    btc['date'] = pd.to_datetime(btc['open_time'], unit='ms')
else:
    btc['date'] = pd.to_datetime(btc['date'])

btc = btc.sort_values('date').reset_index(drop=True)
for c in ['open','high','low','close','volume']:
    if c in btc.columns:
        btc[c] = pd.to_numeric(btc[c], errors='coerce')

N = len(btc)

# Load canonical trades & equity
trades = pd.read_csv(CANONICAL_DIR / 'trades.csv')
eq_df = pd.read_csv(CANONICAL_DIR / 'equity.csv')
eq_col = [c for c in eq_df.columns if 'equity' in c.lower()][0]
eq_all = eq_df[eq_col].astype(float).values[:N]

for c in ['entry_idx','exit_idx']:
    if c in trades.columns:
        trades[c] = trades[c].astype(int)
trades['side'] = trades['side'].astype(str).str.lower()

# compute contrib_all
contrib_all = np.zeros(N)
for _, r in trades.iterrows():
    ex = max(0, min(int(r['exit_idx']), N-1))
    contrib_all[ex] += float(r['R'])

print('Loaded', len(btc), 'candles and', len(trades), 'canonical trades')

Loaded 43778 candles and 3691 canonical trades


In [7]:
# === Klinger oscillator (replicating filter implementation)
high = btc['high']
low = btc['low']
close = btc['close']
volume = btc['volume']

trend = np.where(close > close.shift(1), 1, -1)
trend = pd.Series(trend, index=btc.index)

dm = high - low
dm_prev = dm.shift(1)
cm = dm.where(trend == trend.shift(1), dm_prev).cumsum()

vf = volume * trend * np.abs(2 * (dm / cm - 1))
vf = vf.replace([np.inf, -np.inf], np.nan).fillna(0)

btc['klinger_fast'] = vf.ewm(span=34, adjust=False).mean()
btc['klinger_slow'] = vf.ewm(span=55, adjust=False).mean()
btc['klinger'] = btc['klinger_fast'] - btc['klinger_slow']
btc['klinger_prev'] = btc['klinger'].shift(1)
btc['klinger_prev2'] = btc['klinger'].shift(2)

# === StochRSI (replicating filter implementation)
delta = btc['close'].diff()
up = delta.clip(lower=0)
down = -delta.clip(upper=0)
ma_up = up.ewm(alpha=1/14, adjust=False).mean()
ma_down = down.ewm(alpha=1/14, adjust=False).mean()
btc['rsi14'] = 100 - (100 / (1 + ma_up / ma_down))
STOCH_PER = 14
K_SMOOTH = 3
D_SMOOTH = 3
rsi_min = btc['rsi14'].rolling(STOCH_PER).min()
rsi_max = btc['rsi14'].rolling(STOCH_PER).max()
btc['stochrsi'] = (btc['rsi14'] - rsi_min) / (rsi_max - rsi_min)
btc['k'] = btc['stochrsi'].rolling(K_SMOOTH).mean()
btc['d'] = btc['k'].rolling(D_SMOOTH).mean()
btc['k_prev'] = btc['k'].shift(1)
btc['k_prev2'] = btc['k'].shift(2)
btc['d_prev'] = btc['d'].shift(1)

print('Indicators computed')

Indicators computed


In [8]:
# filter functions
def klinger_above_zero(idx, side):
    try:
        v = btc['klinger_prev'].iat[idx]
    except Exception:
        return False
    if pd.isna(v):
        return False
    return v > 0 if str(side).startswith('l') else v < 0

def stochrsi_cross(idx, side):
    try:
        k1 = btc['k_prev'].iat[idx]
        k2 = btc['k_prev2'].iat[idx]
        d1 = btc['d_prev'].iat[idx]
    except Exception:
        return False
    if pd.isna(k1) or pd.isna(k2) or pd.isna(d1):
        return False
    if str(side).startswith('l'):
        return (k1 > d1 and k2 <= d1)
    else:
        return (k1 < d1 and k2 >= d1)

# Apply combined filter: both must be True at entry_idx
keep = []
for _, r in trades.iterrows():
    idx = max(0, min(int(r['entry_idx']), N-1))
    side = r['side']
    ok = klinger_above_zero(idx, side) and stochrsi_cross(idx, side)
    keep.append(ok)

trades_f = trades.iloc[[i for i,k in enumerate(keep) if k]].copy()
print('Filtered trades:', len(trades_f))

Filtered trades: 1565


In [9]:
# compute filtered contrib and equity
contrib_f = np.zeros(N)
for _, r in trades_f.iterrows():
    ex = max(0, min(int(r['exit_idx']), N-1))
    contrib_f[ex] += float(r['R'])

eq_f = eq_all - np.cumsum(contrib_all - contrib_f)

# save equity CSV and summary
out_eq = OUT_DIR / 'klinger_stochrsi_sl_equity.csv'
pd.DataFrame({'date': btc['date'], 'baseline': eq_all, 'filtered_equity': eq_f}).to_csv(out_eq, index=False)

summary = {
    'n_trades_total': int(len(trades)),
    'n_trades_filtered': int(len(trades_f)),
    'win_rate': float((trades_f['hit'] != 'sl').mean()) if len(trades_f) else None,
    'final_equity_filtered': float(eq_f[-1]),
}
pd.DataFrame([summary]).to_csv(OUT_DIR / 'klinger_stochrsi_sl_summary.csv', index=False)
print('Saved equity to', out_eq)

Saved equity to /Users/juandiegoyoung/Documents/Price_action_and_indicators/experiments/boschonk/BTCUSDT/1h/window_5/rr_1.0/notebooks/klinger_stochrsi_sl_equity.csv


In [ ]:
# === Stop loss distribution for filtered trades
if len(trades_f):
    # ensure numeric columns
    trades_f['entry'] = pd.to_numeric(trades_f['entry'], errors='coerce')
    trades_f['sl'] = pd.to_numeric(trades_f['sl'], errors='coerce')
    trades_f['side'] = trades_f['side'].astype(str).str.lower()

    # absolute distance from entry to stop (price units, positive number)
    def abs_stop(row):
        side = str(row['side']).lower()
        e = row['entry']
        s = row['sl']
        if pd.isna(e) or pd.isna(s):
            return np.nan
        if side.startswith('l'):
            return float(e - s)
        else:
            return float(s - e)

    trades_f['sl_abs'] = trades_f.apply(abs_stop, axis=1)

    # save distribution CSV (absolute stop values)
    out_sl_csv = OUT_DIR / 'klinger_stochrsi_sl_distribution.csv'
    trades_f[['side','entry','sl','sl_abs']].to_csv(out_sl_csv, index=False)

    # plot histogram (price units) limited to 0-3000
    vals = trades_f['sl_abs'].dropna().values
    vals = vals[(vals >= 0) & (vals <= 3000)]
    plt.figure(figsize=(8,5))
    bins = np.linspace(0, 3000, 60)
    plt.hist(vals, bins=bins, color='C0', edgecolor='black', range=(0,3000))
    plt.xlabel('Stop size (price units)')
    plt.ylabel('Count')
    plt.title('Distribución de Stop Loss (filtradas) - valor absoluto (0-3000)')
    plt.xlim(0,3000)
    plt.tight_layout()
    out_hist = OUT_DIR / 'klinger_stochrsi_sl_hist.png'
    plt.savefig(out_hist, dpi=150)
    plt.close()
    print('Saved SL distribution CSV to', out_sl_csv)
    print('Saved SL histogram to', out_hist)
else:
    print('No filtered trades to compute SL distribution')

Saved SL distribution CSV to /Users/juandiegoyoung/Documents/Price_action_and_indicators/experiments/boschonk/BTCUSDT/1h/window_5/rr_1.0/notebooks/klinger_stochrsi_sl_distribution.csv
Saved SL histogram to /Users/juandiegoyoung/Documents/Price_action_and_indicators/experiments/boschonk/BTCUSDT/1h/window_5/rr_1.0/notebooks/klinger_stochrsi_sl_hist.png


In [11]:
# Quick summary printout
print('Summary:')
print(pd.DataFrame([summary]).to_string(index=False))
if len(trades_f):
    print('\nSL distribution (first 10 rows):')
    print(trades_f[['side','entry','sl','sl_abs']].head(10).to_string(index=False))

Summary:
 n_trades_total  n_trades_filtered  win_rate  final_equity_filtered
           3691               1565  0.670927             193.559846

SL distribution (first 10 rows):
 side    entry       sl  sl_abs
 long 23055.98 22811.59  244.39
 long 23296.96 22853.50  443.46
short 23483.11 23974.71  491.60
 long 23979.99 23373.05  606.94
short 22446.39 23945.29 1498.90
 long 23119.47 22655.83  463.64
 long 23721.95 23342.54  379.41
short 23435.95 23635.87  199.92
short 23315.38 23605.07  289.69
 long 23934.52 23481.21  453.31
